In [6]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
from scipy import stats
import pandas as pd
from config import apikey, email
from g_key import gkey
import gmaps
import os

In [7]:
# base url for state codes needed down the line to focus in 
state_url = (f'https://aqs.epa.gov/data/api/list/states?email={email}&key={apikey}')

# get the state codes into a data frame
response=requests.get(state_url)
data=response.json()


In [8]:
# parameter classes
param_url=f'https://aqs.epa.gov/data/api/list/parametersByClass?email={email}&key={apikey}&pc=CRITERIA'

response=requests.get(param_url)
data_param=response.json()


In [9]:
# pulling sample data
# Choose state
state='53'

# choose parameter 
param='42101'

# create the begin date and end date for search using yyyymmdd format
bdate='20200131'
edate='20201231'

# develop the query URL - Sample Data
sample_url=f'https://aqs.epa.gov/data/api/sampleData/byState?email={email}&key={apikey}&param={param}&bdate={bdate}&edate={edate}&state={state}'
response=requests.get(sample_url)
sample_param=response.json()
sample_param['Data'][0].keys()


dict_keys(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc', 'latitude', 'longitude', 'datum', 'parameter', 'date_local', 'time_local', 'date_gmt', 'time_gmt', 'sample_measurement', 'units_of_measure', 'units_of_measure_code', 'sample_duration', 'sample_duration_code', 'sample_frequency', 'detection_limit', 'uncertainty', 'qualifier', 'method_type', 'method', 'method_code', 'state', 'county', 'date_of_last_change', 'cbsa_code'])

In [10]:
#annual summary data - pulling same params from previous kernel - Annual Data by state
annual_url=f'https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={apikey}&param={param}&bdate={bdate}&edate={edate}&state={state}'
response=requests.get(annual_url)
annual_param=response.json()
annual_param['Data'][0].keys()


dict_keys(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc', 'latitude', 'longitude', 'datum', 'parameter', 'sample_duration', 'pollutant_standard', 'metric_used', 'method', 'year', 'units_of_measure', 'event_type', 'observation_count', 'observation_percent', 'validity_indicator', 'valid_day_count', 'required_day_count', 'exceptional_data_count', 'null_observation_count', 'primary_exceedance_count', 'secondary_exceedance_count', 'certification_indicator', 'arithmetic_mean', 'standard_deviation', 'first_max_value', 'first_max_datetime', 'second_max_value', 'second_max_datetime', 'third_max_value', 'third_max_datetime', 'fourth_max_value', 'fourth_max_datetime', 'first_max_nonoverlap_value', 'first_max_n_o_datetime', 'second_max_nonoverlap_value', 'second_max_n_o_datetime', 'ninety_ninth_percentile', 'ninety_eighth_percentile', 'ninety_fifth_percentile', 'ninetieth_percentile', 'seventy_fifth_percentile', 'fiftieth_percentile', 'tenth_percentile', 'local_site_name', 's

In [38]:
# hard coded searches
param='42101'
bdate='20200131'
edate='20201231'

#query URL
query_url=f'https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={apikey}&param={param}&bdate={bdate}&edate={edate}&state='

# response = requests.get(query_url + code).json()


In [39]:
# Set up empty lists for data in each column
states = []
cities = []
lat = []
long = []
mean = []
max_value = []
# Create a list of state codes so our request can loop through and pull each state
state_code = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "15", "16", "17", "18", "19", "20", "21", "22", 
              "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41",
              "42", "44", "45", "46", "47", "48", "49", "50", "51", "53", "54", "55", "56", "66", "72", "78"]
# Loop through each state code and pull mean and max No2 values for all locations in each state
for code in state_code:
    response = requests.get(query_url + code).json()
    try:
        states.extend([x['state'] for x in response['Data']])
        cities.extend([x['city'] for x in response['Data']])
        lat.extend([x['latitude'] for x in response['Data']])
        long.extend([x['longitude'] for x in response['Data']])
        mean.extend([x['arithmetic_mean'] for x in response['Data']])
        max_value.extend([x['first_max_value'] for x in response['Data']])
        
    except: 
        print("This state has no data", code)
        

In [40]:
states_df=pd.DataFrame({
        'State':states,
        'City':cities,
        'Latitude': lat,
        'Longitude':long,
        'Mean Value': mean,
        'Max Value': max_value})

states_df.head(30)

,State,City,Latitude,Longitude,Mean Value,Max Value
0,Alabama,Birmingham,33.553056,-86.815000,0.232099,1.900
1,Alabama,Birmingham,33.553056,-86.815000,0.233456,1.200
2,Alabama,Birmingham,33.521427,-86.844112,0.333946,3.164
3,Alabama,Birmingham,33.521427,-86.844112,0.336042,1.300
4,Alabama,Fairfield,33.485556,-86.915000,0.183963,1.200
5,Alabama,Fairfield,33.485556,-86.915000,0.185756,1.100
6,Alaska,Anchorage,61.205861,-149.824602,0.385518,4.372
7,Alaska,Anchorage,61.205861,-149.824602,0.388289,3.200
8,Alaska,Fairbanks,64.845690,-147.727413,0.322474,2.566
9,Alaska,Fairbanks,64.845690,-147.727413,0.325242,1.900


In [14]:
states_df_grouped=states_df.groupby('State').first()
states_df_grouped

,Latitude,Longitude,Mean Value,Max Value
State,,,,
Alabama,33.553056,-86.815000,0.232099,1.900
Alaska,61.205861,-149.824602,0.385518,4.372
Arizona,33.483780,-112.142560,0.505573,3.800
Arkansas,34.756189,-92.281296,0.286346,1.300
California,33.830620,-117.938450,0.339064,2.100
Colorado,38.830895,-104.839243,0.324609,1.853
Connecticut,41.821342,-73.297257,0.148332,4.142
Delaware,39.739444,-75.558056,0.223580,1.000
District Of Columbia,38.921847,-77.013178,0.238271,2.007


In [42]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
latlong = states_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = states_df["Max Value"].astype(float)

In [43]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(latlong, weights=states_df['Max Value'],
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

#save figure
# fig.savefig('2020_CO2_Heatmap.png')

Figure(layout=FigureLayout(height='420px'))

In [47]:
locations=states_df[["Latitude", "Longitude"]].astype(float)
monitor_layer=gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,info_box_content=[f"Monitor location:{city}" for city in cities])

fig=gmaps.figure()
fig.add_layer(monitor_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# hard coded searches
param='42101'
bdate19='20190101'
edate19='20191231'

#query URL
query_url=f'https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={apikey}&param={param}&bdate={bdate19}&edate={edate19}&state='

# response = requests.get(query_url + code).json()

In [18]:
# Set up empty lists for data in each column
states19 = []
lat19 = []
long19 = []
mean19 = []
max_value19 = []
# Create a list of state codes so our request can loop through and pull each state
state_code = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "15", "16", "17", "18", "19", "20", "21", "22", 
              "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41",
              "42", "44", "45", "46", "47", "48", "49", "50", "51", "53", "54", "55", "56", "66", "72", "78"]
# Loop through each state code and pull mean and max No2 values for all locations in each state
for code in state_code:
    response = requests.get(query_url + code).json()
    try:
        states19.extend([x['state'] for x in response['Data']])
        lat19.extend([x['latitude'] for x in response['Data']])
        long19.extend([x['longitude'] for x in response['Data']])
        mean19.extend([x['arithmetic_mean'] for x in response['Data']])
        max_value19.extend([x['first_max_value'] for x in response['Data']])
        
    except: 
        print("This state has no data", code)

In [19]:
# put into data frame
states_19_df=pd.DataFrame({'State':states19,
           'Latitude': lat19,
           'Longitude':long19,
           'Mean Value': mean19,
           'Max Value': max_value19})

states_19_df.head(30)

,State,Latitude,Longitude,Mean Value,Max Value
0,Alabama,33.553056,-86.815000,0.418478,2.400
1,Alabama,33.553056,-86.815000,0.420199,1.100
2,Alabama,33.521427,-86.844112,0.263202,1.794
3,Alabama,33.521427,-86.844112,0.267074,1.300
4,Alabama,33.485556,-86.915000,0.204744,1.800
5,Alabama,33.485556,-86.915000,0.207541,1.400
6,Alaska,61.205861,-149.824602,0.534130,3.463
7,Alaska,61.205861,-149.824602,0.533726,2.600
8,Alaska,64.845690,-147.727413,0.378840,3.547
9,Alaska,64.845690,-147.727413,0.378499,2.400


In [20]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
latlong = states_19_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = states_19_df["Max Value"].astype(float)

In [21]:
# Plot Heatmap
fig19 = gmaps.figure()

# Create heat layer
heat_layer19 = gmaps.heatmap_layer(latlong, weights=states_19_df['Max Value'],
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig19.add_layer(heat_layer19)

# Display figure
fig19

#save fig
# fig19.savefig('2019_CO2_heatmap.png')


Figure(layout=FigureLayout(height='420px'))

In [22]:
# hard coded searches
param='42101'
bdate18='20180101'
edate18='20181231'

#query URL
query_url=f'https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={apikey}&param={param}&bdate={bdate18}&edate={edate18}&state='

# response = requests.get(query_url + code).json()

In [23]:
# Set up empty lists for data in each column
states18 = []
lat18 = []
long18 = []
mean18 = []
max_value18 = []
# Create a list of state codes so our request can loop through and pull each state
state_code = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "15", "16", "17", "18", "19", "20", "21", "22", 
              "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41",
              "42", "44", "45", "46", "47", "48", "49", "50", "51", "53", "54", "55", "56", "66", "72", "78"]
# Loop through each state code and pull mean and max No2 values for all locations in each state
for code in state_code:
    response = requests.get(query_url + code).json()
    try:
        states18.extend([x['state'] for x in response['Data']])
        lat18.extend([x['latitude'] for x in response['Data']])
        long18.extend([x['longitude'] for x in response['Data']])
        mean18.extend([x['arithmetic_mean'] for x in response['Data']])
        max_value18.extend([x['first_max_value'] for x in response['Data']])
        
    except: 
        print("This state has no data", code)

In [29]:
# put into data frame
states_18_df=pd.DataFrame({'State':states18,
           'Latitude': lat18,
           'Longitude':long18,
           'Mean Value': mean18,
           'Max Value': max_value18})

states_18_df.head(30)

,State,Latitude,Longitude,Mean Value,Max Value
0,Alabama,33.553056,-86.815000,0.220259,1.718
1,Alabama,33.553056,-86.815000,0.217866,1.100
2,Alabama,33.521427,-86.844112,0.341005,1.940
3,Alabama,33.521427,-86.844112,0.344179,1.600
4,Alabama,33.485556,-86.915000,0.198432,1.800
5,Alabama,33.485556,-86.915000,0.200850,1.200
6,Alaska,61.205861,-149.824602,0.526335,4.599
7,Alaska,61.205861,-149.824602,0.526145,2.900
8,Alaska,64.845690,-147.727413,0.345062,4.469
9,Alaska,64.845690,-147.727413,0.347891,2.000


In [30]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
latlong = states_18_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = states_18_df["Max Value"].astype(float)

In [31]:
# Plot Heatmap
fig18 = gmaps.figure()

# Create heat layer
heat_layer18 = gmaps.heatmap_layer(latlong, weights=states_18_df['Max Value'],
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig18.add_layer(heat_layer18)

# Display figure
fig18

#save fig
# fig19.savefig('2019_CO2_heatmap.png')

Figure(layout=FigureLayout(height='420px'))

In [32]:
# hard coded searches
param='42101'
bdate17='20170101'
edate17='20171231'

#query URL
query_url=f'https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={apikey}&param={param}&bdate={bdate17}&edate={edate17}&state='

# response = requests.get(query_url + code).json()

In [33]:
# Set up empty lists for data in each column
states17 = []
lat17 = []
long17 = []
mean17 = []
max_value17 = []
# Create a list of state codes so our request can loop through and pull each state
state_code = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "15", "16", "17", "18", "19", "20", "21", "22", 
              "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41",
              "42", "44", "45", "46", "47", "48", "49", "50", "51", "53", "54", "55", "56", "66", "72", "78"]
# Loop through each state code and pull mean and max No2 values for all locations in each state
for code in state_code:
    response = requests.get(query_url + code).json()
    try:
        states17.extend([x['state'] for x in response['Data']])
        lat17.extend([x['latitude'] for x in response['Data']])
        long17.extend([x['longitude'] for x in response['Data']])
        mean17.extend([x['arithmetic_mean'] for x in response['Data']])
        max_value17.extend([x['first_max_value'] for x in response['Data']])
        
    except: 
        print("This state has no data", code)

In [34]:
# put into data frame
states_17_df=pd.DataFrame({'State':states17,
           'Latitude': lat17,
           'Longitude':long17,
           'Mean Value': mean17,
           'Max Value': max_value17})

states_17_df.head(30)

,State,Latitude,Longitude,Mean Value,Max Value
0,Alabama,33.553056,-86.815000,0.317434,2.933
1,Alabama,33.553056,-86.815000,0.319815,2.300
2,Alabama,33.521427,-86.844112,0.364436,1.640
3,Alabama,33.521427,-86.844112,0.367024,1.300
4,Alabama,33.485556,-86.915000,0.190410,1.600
5,Alabama,33.485556,-86.915000,0.192860,1.100
6,Alaska,61.205861,-149.824602,0.682745,4.712
7,Alaska,61.205861,-149.824602,0.681824,3.700
8,Alaska,64.845690,-147.727413,0.363771,3.634
9,Alaska,64.845690,-147.727413,0.365459,2.300


In [35]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
latlong = states_17_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = states_17_df["Max Value"].astype(float)

In [36]:
# Plot Heatmap
fig18 = gmaps.figure()

# Create heat layer
heat_layer17 = gmaps.heatmap_layer(latlong, weights=states_17_df['Max Value'],
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig17.add_layer(heat_layer18)

# Display figure
fig18

#save fig
# fig19.savefig('2019_CO2_heatmap.png')

NameError: name 'fig17' is not defined

In [25]:
file_path="Net_generation_for_all_sectors (1).csv"
df = pd.read_csv(file_path)
df

,Net generation for all sectors,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241
0,https://www.eia.gov/electricity/data/browser/#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Jan 30 2021 12:20:00 GMT-0700 (Mountain St...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Source: U.S. Energy Information Administration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,description,units,source key,Jan-01,Feb-01,Mar-01,Apr-01,May-01,Jun-01,Jul-01,...,Feb-20,Mar-20,Apr-20,May-20,Jun-20,Jul-20,Aug-20,Sep-20,Oct-20,Nov-20
4,Net generation for all sectors,thousand megawatthours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,Hawaii : hydro-electric pumped storage,thousand megawatthours,ELEC.GEN.HPS-HI-99.M,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1303,Hawaii : other,thousand megawatthours,ELEC.GEN.OTH-HI-99.M,12,12,5,15,17,19,18,...,25,30,23,27,21,30,28,21,19,31
1304,Hawaii : all solar,thousand megawatthours,ELEC.GEN.TSN-HI-99.M,--,--,--,--,--,--,--,...,119,136,153,167,164,168,167,155,140,123
1305,Hawaii : small-scale solar photovoltaic,thousand megawatthours,ELEC.GEN.DPV-HI-99.M,--,--,--,--,--,--,--,...,87,99,107,114,113,116,115,106,99,87
